In [2]:
import os
import csv
import io
import json
from uuid import uuid4
from genson import SchemaBuilder
# from flatten_json import flatten, unflatten
# from datamodel_code_generator import DataModelType, PythonVersion
# from datamodel_code_generator.model import get_data_model_types
# from datamodel_code_generator.parser.jsonschema import JsonSchemaParser


def json_to_json_schema(input_json):
    input_dict = input_json.data
    builder = SchemaBuilder()
    builder.add_object(input_dict)
    json_schema = builder.to_schema()
    return json_schema

In [54]:
test_string = 'uid,Name,Array,Type,Required,Default\r\n0c5549e5-40f0-4b2b-b993-da60eb594cd6,hello,True,string,True,efff2\nf7c38eba-5cbb-49c9-9719-ffcb12b0be6b,goodbye.lovers,True,boolean,True,3412341234\n96c3f8b4-6ad4-48af-bd60-352801fb035f,goodbye.friends,True,number,True,12345\n2673f249-f945-4b9e-ac40-10a98ffdc489,fu,False,,True,\r\n627f9e99-0c1d-458a-b8ec-64aeaebe913c,bar,False,object,True,\r\n'


In [73]:
all_errors

['Missing datatype on row 4', 'Type and Default mismatch on row 2']

True